<a href="https://colab.research.google.com/github/gytjd/QA_Data_Augmentation/blob/main/Quantuam_AI_%EC%9E%90%EB%8F%99%EC%B0%A8_%EB%B2%88%ED%98%B8%ED%8C%90_%EC%9D%B8%EC%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# 이미지 전처리

import cv2
import numpy as np
from scipy import ndimage
from statistics import median


def process_image(img_path, save_path):
    # 이미지 로딩
    img = cv2.imread(img_path)

    # 1. 이미지를 grayscale로 변환
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 2. 이미지 노이즈 제거
    denoised = cv2.fastNlMeansDenoising(gray, None, 30, 7, 21)

    # 3. 가우시안 블러링 실행
    blurred = cv2.GaussianBlur(denoised, (5,5), 0)

    # 4. 이미지의 엣지검출
    edges = cv2.Canny(blurred, 50, 150)

    # 5. 확률적 허프변환에 의한 선분 검출
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, 50, minLineLength=50, maxLineGap=5)

    # 6. 선의 유무에 따라
    if lines is not None:
        angles = []
        for line in lines:
            x1, y1, x2, y2 = line[0]
            # 6-2-1. 추출된 선들 기반으로 각도 계산
            angle = np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi
            angles.append(angle)

        # 6-2-2. 각도 계산에 사용된 선의 길이가 선들의 중앙값보다 작은 각도 제외
        med_angle = median(angles)
        filtered_angles = [angle for angle in angles if abs(angle) > med_angle]

        # 6-2-3. 남은 각도의 분포 기반으로 수정각도 계산
        correct_angle = np.mean(filtered_angles)

        # 6-2-4. 수정각도가 0 또는 90이 아닐때 수정각도만큼 이미지 회전
        if correct_angle not in [0, 90]:
            rotated = ndimage.rotate(img, correct_angle)
        else:
            rotated = img.copy()
    else:
        # 6-1. 각도 계산이 불가능하므로 해당 이미지대로 저장
        rotated = img.copy()

    # 7. 학습을 위해 이미지의 가로길이, 세로길이, DPI를 일관되게 저장
    resized = cv2.resize(rotated, (300, 300))
    cv2.imwrite("save_path.jpg", resized)
    return resized

# 데이터 augmentation
def augment_image(image):
    # 1. 랜덤 회전
    angle = np.random.randint(-15, 15)
    rotated = ndimage.rotate(image, angle, reshape=False, mode='nearest')

    # 2. 랜덤 확대/축소
    scale_factor = np.random.uniform(0.9, 1.1)
    resized = cv2.resize(rotated, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_LINEAR)

    # 3. 랜덤 밝기 조절
    brightness = np.random.randint(-50, 50)
    brightened = cv2.convertScaleAbs(resized, alpha=1, beta=brightness)

    # 4. 랜덤 대비 조절
    contrast = np.random.uniform(0.8, 1.2)
    contrasted = cv2.convertScaleAbs(brightened, alpha=contrast, beta=0)

    # 5. 랜덤 플립 (수평)
    if np.random.rand() > 0.5:
        flipped = cv2.flip(image, 1)
    else:
        flipped = image.copy()

    # 6. 랜덤 노이즈 추가
    mean = 0
    stddev = 25
    noise = np.random.normal(mean, stddev, image.shape).astype(np.uint8)
    noisy = cv2.add(image, noise)

    return noisy

def save_augmented_images(original_path, save_dir, num_augments=1):
    original = process_image(original_path, None)
    for i in range(num_augments):
        augmented = augment_image(original)
        save_path = f"{save_dir}/augmented_{i}_{original_path.split('/')[-1]}"
        cv2.imwrite(save_path, augmented)

# 이미지 경로와 저장 디렉토리를 지정하여 함수 호출
save_augmented_images('/content/data_input/numberpanel3.png', '/content/data_output_augmented/')
